In [ ]:
# pyright: reportMissingImports=false
# pyright: reportMissingModuleSource=false

import uuid
import random
import hashlib
import os
import sys
import time
import logging
import datetime
import json
from datetime import datetime, timedelta
import yaml
import pytest
import importlib
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import requests
import pandas_gbq
from sklearn.model_selection import ParameterGrid, ParameterSampler
from scipy.signal import argrelextrema
from dreams_core.googlecloud import GoogleCloud as dgc
from dreams_core import core as dc
import matplotlib.pyplot as plt
import seaborn as sns
import progressbar


# load dotenv
load_dotenv()


# import local files if necessary
# pyright: reportMissingImports=false
sys.path.append('..//src')
import training_data as td
importlib.reload(td)
import feature_engineering as fe
importlib.reload(fe)
import coin_wallet_metrics as cwm
importlib.reload(cwm)
import modeling as m
importlib.reload(m)
import insights as i
importlib.reload(i)
import utils as u
importlib.reload(u)


# configure logger
logger = dc.setup_logger()
logger.setLevel(logging.DEBUG)

# Custom format function for displaying numbers
pd.set_option('display.float_format', lambda x: f'{x:.12g}')
# pd.reset_option('display.float_format')


# Load all configs as global variables
global CONFIG, METRICS_CONFIG, MODELING_CONFIG, EXPERIMENTS_CONFIG, MODELING_FOLDER
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
CONFIG = config
METRICS_CONFIG = metrics_config
MODELING_CONFIG = modeling_config
EXPERIMENTS_CONFIG = experiments_config
MODELING_FOLDER = MODELING_CONFIG['modeling']['modeling_folder']
modeling_folder = MODELING_FOLDER

## Overall Sequencing

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
importlib.reload(u)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
logger.setLevel(logging.INFO)


start_date = config['training_data']['training_period_start']
end_date = config['training_data']['modeling_period_end']

# Retrieve market data
market_data_df = td.retrieve_market_data()
market_data_df, _ = cwm.split_dataframe_by_coverage(market_data_df, start_date, end_date, id_column='coin_id')
prices_df = market_data_df[['coin_id','date','price']].copy()


In [ ]:
if 'profits_df' not in globals():
    profits_df = None

profits_df = i.rebuild_profits_df_if_necessary(
                config,
                modeling_folder,
                prices_df,
                profits_df)


# remove records from market_data_df that don't have transfers if configured to do so
if config['data_cleaning']['exclude_coins_without_transfers']:
    market_data_df = market_data_df[market_data_df['coin_id'].isin(profits_df['coin_id'])]
    prices_df = market_data_df[['coin_id','date','price']].copy()



In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
importlib.reload(u)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
logger.setLevel(logging.INFO)



In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
importlib.reload(u)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
logger.setLevel(logging.INFO)


X_train, X_test, y_train, y_test, performance_df = i.build_configured_model_input(
                                    profits_df,
                                    market_data_df,
                                    config,
                                    metrics_config,
                                    modeling_config)

# 3.4 Train the model using the current configuration and log the results
model, model_id = m.train_model(
                    X_train,
                    y_train,
                    modeling_folder,
                    modeling_config['modeling']['model_params'])

# 3.5 Evaluate and save the model's performance on the test set to a CSV
metrics_df = m.evaluate_model(model, X_test, y_test, model_id, modeling_folder)

In [ ]:
performance_df

In [ ]:
import pandas as pd
import logging

logger = logging.getLogger(__name__)



def calculate_mooncrater_targets(performance_df, modeling_config):
    """
    Calculates 'is_moon' and 'is_crater' target variables based on performance.

    Parameters:
    - performance_df: DataFrame with columns 'coin_id' and 'performance'.
    - modeling_config: Configuration for modeling with target variable thresholds.

    Returns:
    - target_variables_df: DataFrame with columns 'coin_id', 'is_moon', and 'is_crater'.
    """
    moon_threshold = modeling_config['target_variables']['moon_threshold']
    crater_threshold = modeling_config['target_variables']['crater_threshold']
    moon_minimum_percent = modeling_config['target_variables']['moon_minimum_percent']
    crater_minimum_percent = modeling_config['target_variables']['crater_minimum_percent']

    target_variables_df = performance_df.copy()
    target_variables_df['is_moon'] = (target_variables_df['performance'] >= moon_threshold).astype(int)
    target_variables_df['is_crater'] = (target_variables_df['performance'] <= crater_threshold).astype(int)

    total_coins = len(target_variables_df)
    moons = target_variables_df['is_moon'].sum()
    craters = target_variables_df['is_crater'].sum()

    # Ensure minimum percentage for moons and craters
    if moons / total_coins < moon_minimum_percent:
        additional_moons_needed = int(total_coins * moon_minimum_percent) - moons
        moon_candidates = target_variables_df[target_variables_df['is_moon'] == 0].nlargest(additional_moons_needed, 'performance')
        target_variables_df.loc[moon_candidates.index, 'is_moon'] = 1

    if craters / total_coins < crater_minimum_percent:
        additional_craters_needed = int(total_coins * crater_minimum_percent) - craters
        crater_candidates = target_variables_df[target_variables_df['is_crater'] == 0].nsmallest(additional_craters_needed, 'performance')
        target_variables_df.loc[crater_candidates.index, 'is_crater'] = 1

    return target_variables_df[['coin_id', 'is_moon', 'is_crater']]


In [ ]:
importlib.reload(fe)
target_variables_df, performance_df, outcomes_df = fe.create_target_variables(market_data_df, config['training_data'], modeling_config)

In [ ]:
performance_df

## Junkyard

## tests failing

In [ ]:
importlib.reload(td)
importlib.reload(cwm)
importlib.reload(fe)
importlib.reload(m)
importlib.reload(i)
importlib.reload(u)
config = u.load_config('../config/config.yaml')
metrics_config = u.load_config('../config/metrics_config.yaml')
modeling_config = u.load_config('../config/modeling_config.yaml')
experiments_config = u.load_config('../config/experiments_config.yaml')
logger.setLevel(logging.INFO)




def valid_prices_df():
    """
    Fixture to create a sample DataFrame with valid price data for multiple coins.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 2,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5, 35000, 2500, 0.6]
    })
valid_prices_df=valid_prices_df()

def valid_training_data_config():
    """
    Fixture to create a sample training data configuration.
    """
    return {
        'modeling_period_start': '2023-01-01',
        'modeling_period_end': '2023-12-31'
    }
valid_training_data_config=valid_training_data_config()

def no_change_prices_df():
    """
    Fixture to create a sample DataFrame with no price change for some coins.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 2,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5, 30000, 2500, 0.5]
    })
no_change_prices_df=no_change_prices_df()

def negative_performance_prices_df():
    """
    Fixture to create a sample DataFrame with negative performance for some coins.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 2,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5, 25000, 2500, 0.4]
    })
negative_performance_prices_df=negative_performance_prices_df()

def multiple_datapoints_prices_df():
    """
    Fixture to create a sample DataFrame with multiple data points between start and end dates.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 4,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01',
                 '2023-06-15', '2023-06-15', '2023-06-15',
                 '2023-09-30', '2023-09-30', '2023-09-30',
                 '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5,
                  32000, 2200, 0.55,
                  34000, 2400, 0.58,
                  35000, 2500, 0.6]
    })
multiple_datapoints_prices_df=multiple_datapoints_prices_df()


"""
Test prepare_and_compute_performance function with multiple data points between start and end dates.

This test ensures that the function correctly calculates performance using only start and end dates,
ignoring intermediate data points.
"""
performance_df, outcomes_df = fe.prepare_and_compute_performance(multiple_datapoints_prices_df, valid_training_data_config)

expected_performance = pd.DataFrame({
    'coin_id': ['BTC', 'ETH', 'XRP'],
    'performance': [0.1667, 0.25, 0.2]
})

assert (np.isclose(performance_df['performance'].values, expected_performance['performance'].values, rtol=1e-4, atol=1e-4)).all()

expected_outcomes = pd.DataFrame({
    'coin_id': ['BTC', 'ETH', 'XRP'],
    'outcome': ['performance calculated'] * 3
})

assert np.array_equal(outcomes_df.values, expected_outcomes.values)

In [ ]:
assert np.all(np.isclose(performance_df['performance'].values, expected_performance['performance'].values, rtol=1e-4, atol=1e-4))


In [ ]:
expected_performance

In [ ]:

def negative_performance_prices_df():
    """
    Fixture to create a sample DataFrame with negative performance for some coins.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 2,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01', '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5, 25000, 2500, 0.4]
    })
negative_performance_prices_df=negative_performance_prices_df()

def test_prepare_and_compute_performance_negative(negative_performance_prices_df, valid_training_data_config):
    """
    Test prepare_and_compute_performance function with negative performance for some coins.

    This test ensures that the function correctly calculates negative performance values
    for coins with price decreases and correct performance for others.
    """
    performance_df, outcomes_df = fe.prepare_and_compute_performance(negative_performance_prices_df, valid_training_data_config)

    expected_performance = pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'],
        'performance': [-0.1667, 0.25, -0.2]
    })

    assert np.array_equal(performance_df.values, expected_performance.values)

    for actual, expected in zip(performance_df['performance'], expected_performance['performance']):
        assert actual == pytest.approx(expected, abs=1e-4)

@pytest.fixture
def multiple_datapoints_prices_df():
    """
    Fixture to create a sample DataFrame with multiple data points between start and end dates.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 4,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01',
                 '2023-06-15', '2023-06-15', '2023-06-15',
                 '2023-09-30', '2023-09-30', '2023-09-30',
                 '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5,
                  32000, 2200, 0.55,
                  34000, 2400, 0.58,
                  35000, 2500, 0.6]
    })

@pytest.mark.unit
def test_prepare_and_compute_performance_multiple_datapoints(multiple_datapoints_prices_df, valid_training_data_config):
    """
    Test prepare_and_compute_performance function with multiple data points between start and end dates.

    This test ensures that the function correctly calculates performance using only start and end dates,
    ignoring intermediate data points.
    """
    performance_df, outcomes_df = fe.prepare_and_compute_performance(multiple_datapoints_prices_df, valid_training_data_config)

    expected_performance = pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'],
        'performance': [0.1667, 0.25, 0.2]
    })

    assert np.array_equal(performance_df.values, expected_performance.values)

    for actual, expected in zip(performance_df['performance'], expected_performance['performance']):
        assert actual == pytest.approx(expected, abs=1e-4)

In [ ]:
def test_prepare_and_compute_performance_negative(negative_performance_prices_df, valid_training_data_config):
    """
    Test prepare_and_compute_performance function with negative performance for some coins.

    This test ensures that the function correctly calculates negative performance values
    for coins with price decreases and correct performance for others.
    """
    performance_df, outcomes_df = fe.prepare_and_compute_performance(negative_performance_prices_df, valid_training_data_config)

    expected_performance = pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'],
        'performance': [-0.1667, 0.25, -0.2]
    })

    assert np.array_equal(performance_df.values, expected_performance.values)

    for actual, expected in zip(performance_df['performance'], expected_performance['performance']):
        assert actual == pytest.approx(expected, abs=1e-4)

@pytest.fixture
def multiple_datapoints_prices_df():
    """
    Fixture to create a sample DataFrame with multiple data points between start and end dates.
    """
    return pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'] * 4,
        'date': ['2023-01-01', '2023-01-01', '2023-01-01',
                 '2023-06-15', '2023-06-15', '2023-06-15',
                 '2023-09-30', '2023-09-30', '2023-09-30',
                 '2023-12-31', '2023-12-31', '2023-12-31'],
        'price': [30000, 2000, 0.5,
                  32000, 2200, 0.55,
                  34000, 2400, 0.58,
                  35000, 2500, 0.6]
    })

@pytest.mark.unit
def test_prepare_and_compute_performance_multiple_datapoints(multiple_datapoints_prices_df, valid_training_data_config):
    """
    Test prepare_and_compute_performance function with multiple data points between start and end dates.

    This test ensures that the function correctly calculates performance using only start and end dates,
    ignoring intermediate data points.
    """
    performance_df, outcomes_df = fe.prepare_and_compute_performance(multiple_datapoints_prices_df, valid_training_data_config)

    expected_performance = pd.DataFrame({
        'coin_id': ['BTC', 'ETH', 'XRP'],
        'performance': [0.1667, 0.25, 0.2]
    })

    assert np.array_equal(performance_df.values, expected_performance.values)

    for actual, expected in zip(performance_df['performance'], expected_performance['performance']):
        assert actual == pytest.approx(expected, abs=1e-4)